In [68]:
import os
import json
import numpy as np
import xml.etree.ElementTree as ET

annopath = 'Annotations'
set_file = 'train.txt'
label_map_path = 'label_map.json'

coco_annotaton = {}
images_list = []
annotations_list = []
categories_list = []

with open(set_file,'r') as f:
    file_list = list(map(lambda x:x.strip(), f.readlines()))

with open(label_map_path,'r') as fp:
    label_map = json.load(fp)

categories_list = label_map
coco_annotaton['categories'] = categories_list
categories_dict = {cate['name']:cate['id'] for cate in categories_list}

def recursive_parse_xml_to_dict(xml):
  if not xml:
    return {xml.tag: xml.text}
  result = {}
  for child in xml:
    child_result = recursive_parse_xml_to_dict(child)
    if child.tag != 'object':
      result[child.tag] = child_result[child.tag]
    else:
      if child.tag not in result:
        result[child.tag] = []
      result[child.tag].append(child_result[child.tag])
  return {xml.tag: result}

for fname in file_list:
    print(fname)
    try:
        with open(os.path.join(annopath,fname+'.xml'),'r') as fid:
            xml_root = ET.parse(fid).getroot()
            root_anno = recursive_parse_xml_to_dict(xml_root)['annotation']
            try:
                filename = root_anno['filename']
                size = root_anno['size']
            except:
                continue
            img_id = len(images_list)
            images_list.append({'id':img_id,
                            'file_name':os.path.split(filename)[-1],
                            'width':int(size['width']),
                            'height':int(size['height'])})
            try:
                objects = root_anno['object']
            except:
                continue
            for obj in objects:
                bndbox = obj['bndbox']
                x = float(bndbox['xmin'])
                y = float(bndbox['ymin'])
                w = float(bndbox['xmax'])-float(bndbox['xmin'])
                h = float(bndbox['ymax'])-float(bndbox['ymin'])
                annotations_list.append({'id':len(annotations_list),
                                         'image_id':img_id,
                                         'category_id':categories_dict[obj['name']],
                                         'bbox':[x,y,w,h],
                                         'iscrowd':0,
                                         'area':w*h*np.pi*.25})
    except FileNotFoundError:
        print('No file \'%s\' found!'%os.path.join(annopath,fname+'.xml'))
coco_annotaton['images'] = images_list
coco_annotaton['annotations'] = annotations_list

with open('demo_annotation.json','w') as fw:
    json.dump(coco_annotaton,fw)

420_2018101875717_1
420_2018101875718
420_2018101875724
420_2018101875721
420_2018101875719
420_2018101875720
420_2018101875723
420_2018101875722
420_2018101875725
420_2018101875717
420_2018101875718_1
11
No file 'Annotations/11.xml' found!


In [66]:
# 制作一个玩具result用于测试

import random
import numpy as np

annos = coco_annotaton['annotations']
result_list = []
for box in annos:
    for i in range(50):
        imgid = box['image_id']
        cateid = box['category_id']
        bbox = list(np.array(box['bbox'])*random.uniform(0.9,1.1))
        score = random.uniform(.5,1.) if random.random()>.5 else random.random()
        result_list.append({'image_id':imgid,
                            'category_id':cateid,
                            'bbox':bbox,
                            'score':score})

with open('result.json','w') as fw:
    json.dump(result_list,fw)

In [67]:
np.pi*.25

0.7853981633974483